<a href="https://colab.research.google.com/github/mariasakorafou/maria1/blob/main/Askisis%20final%20version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#εγκατάσταση πακέτου PyOD (https://pyod.readthedocs.io/en/latest/)
!pip install pyod
!pip install ppscore matplotlib seaborn pandas
!pip install --upgrade xgboost scikit-learn

In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier





In [ ]:
# Load the dataset

file_path = "https://raw.githubusercontent.com/kdemertzis/EKPA/refs/heads/main/Data/DarkNet.csv"
dataset = pd.read_csv(file_path)


<ipython-input-50-4306e067de65>:4: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(file_path)


In [ ]:
dataset = dataset.drop(columns=[col for col in ['Unnamed: 1', 'Label-1.1', 'Unnamed: 3', 'Unnamed: 5', 'Unnamed: 6'] if col in dataset.columns])

In [ ]:


# Normalize 'Label-2'
dataset['Label-2'] = dataset['Label-2'].str.upper().str.replace('-', '_')

# Convert 'Label-1' to binary (Tor: 1, Non-Tor: 0)
dataset['Label-1'] = dataset['Label-1'].apply(lambda x: 1 if x.upper() == 'TOR' else 0)



In [ ]:
# Oversampling the minority class (Tor)
tor_samples = dataset[dataset['Label-1'] == 1]
non_tor_samples = dataset[dataset['Label-1'] == 0]




In [ ]:
# Manually oversample Tor to balance the dataset
oversampled_tor = tor_samples.sample(n=len(non_tor_samples), replace=True, random_state=42)
balanced_dataset = pd.concat([non_tor_samples, oversampled_tor]).sample(frac=1, random_state=42).reset_index(drop=True)


In [ ]:

# Prepare features and labels
X = balanced_dataset['Label-2'].str.get_dummies()
y = balanced_dataset['Label-1']



In [ ]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


In [ ]:
# Random Forest Classifier
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [ ]:

# Evaluate Random Forest
rf_predictions = rf_model.predict(X_test)
rf_report = classification_report(y_test, rf_predictions)
rf_conf_matrix = confusion_matrix(y_test, rf_predictions)
print("Random Forest Classification Report:\n", rf_report)
print("Random Forest Confusion Matrix:\n", rf_conf_matrix)


Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.62      0.65     20157
           1       0.65      0.71      0.68     20156

    accuracy                           0.67     40313
   macro avg       0.67      0.67      0.66     40313
weighted avg       0.67      0.67      0.66     40313

Random Forest Confusion Matrix:
 [[12569  7588]
 [ 5902 14254]]


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="xgboost")

# XGBoost Classifier
xgb_model = XGBClassifier(eval_metric='logloss', random_state=42)
xgb_model.fit(X_train, y_train)

# Αξιολόγηση του XGBoost
y_pred_xgb = xgb_model.predict(X_test)
xgb_report = classification_report(y_test, y_pred_xgb)
xgb_conf_matrix = confusion_matrix(y_test, y_pred_xgb)

# Εκτύπωση αποτελεσμάτων
print("XGBoost Classification Report:\n", xgb_report)
print("XGBoost Confusion Matrix:\n", xgb_conf_matrix)

XGBoost Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.62      0.65     20157
           1       0.65      0.71      0.68     20156

    accuracy                           0.67     40313
   macro avg       0.67      0.67      0.66     40313
weighted avg       0.67      0.67      0.66     40313

XGBoost Confusion Matrix:
 [[12569  7588]
 [ 5902 14254]]
